In [ ]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

#- Connect to MonogoDB
try:
    #- Connect to the MongoDB instance
    connection = MongoClient("mongodb://localhost:27017")
    #- Database name
    database = connection['sentiments']
    #- Collection name
    comments_collection = database['comments']
except:
    print("[Error]: Failed to connect to database..")
    connection = None

#- Définir l'URL du site
url="https://www.hespress.com/%d8%a5%d8%af%d8%a7%d8%b1%d8%a9-%d8%a7%d9%84%d9%85%d8%ac%d8%a7%d9%84-%d8%a7%d9%84%d8%ac%d9%88%d9%8a-%d9%88%d8%aa%d8%b1%d8%b3%d9%8a%d9%85-%d8%a7%d9%84%d8%ad%d8%af%d9%88%d8%af-%d9%88%d8%a7%d9%84%d8%ac-1119029.html"
    
#- Création d'une requête GET
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
response = requests.get(url, headers = header)

#- On récupére le contenu HTML
soup = BeautifulSoup(response.text, "html.parser")

#- Extraire les éléments contenant la classe "comments"
comments = soup.find_all("div", class_="comments")

#- Initaliser un tableau dont on stocke le scrap des commentaires.
comments_to_insert = []

#- Itérer les commentaires pour extraire le contenu
for comment in comments:
    for ul in comment.find_all('ul'):
        for li in ul.find_all('li'):
            div_container = li.find('div', {'class': 'comment-body'})
            div1 = div_container.find('div', {'class': 'comment-head'})
            div2 = div_container.find('div', {'class': 'comment-text'})
            comment_obj = {"author": div1.text.split("\n")[2], "text": div2.text}
            comments_to_insert.append(comment_obj)
            
#- Insertion des données dans notre collection 'commentaire' en MongoDB
if len(comments_to_insert) > 0:
    comments_collection.insert_many(comments_to_insert)